# Outline

- train word embeddings by using `matichon.json`
- use gensim `Word2Vec` model
- find similar words, calculate cosine similarity

# Import 

In [1]:
import pandas as pd
import numpy as np
import re

from gensim.models import Word2Vec

from pythainlp.tokenize import word_tokenize

- custom tokenization function
- remove all quatations and shrink newlines `\n` and white spaces 

In [2]:
def my_tokenize(text):
    text = re.sub(r'[\"\']', '', text) # remove quotations
    text = re.sub(r'[\n\t\s]', ' ', text) # shrink newlines/spaces
    tokens = word_tokenize(text, keep_whitespace=True)
    return tokens

# Load data

- need only article, drop the other columns
- tokenize article and store as list of tokens

In [3]:
df = pd.read_json('data/matichon.json')
df

,headline,article,date,category,url,id
0,ซาอุฯจ่อเปิดไฟเขียวให้สิทธิหญิงม่าย-หย่าร้างปก...,(2 ธ.ค.58) หนังสือพิมพ์อัล ริยาดของทางการซาอุด...,2015-12-04 03:35:18,foreign,https://www.matichon.co.th/foreign/news_293,293
1,"""ไก่อู""ชี้ ตู่-เต้น ไม่ได้มีหน้าที่ตรวจสอบทุจร...","""บิ๊กป้อม"" แจง ครม. มีความพยายามยุยงปลุกปั่นให...",2015-12-04 04:10:49,politics,https://www.matichon.co.th/politics/news_329,329
2,"เปิดใจ ""โบว์ แวนดา"" ระหว่างรอยิ้มได้เต็มที่ในว...",แม้จะทำหน้าที่ภรรยาที่ดีมาเฝ้าปอ – ทฤษฎี สหวงษ...,2015-12-04 06:30:11,entertainment,https://www.matichon.co.th/entertainment/news_375,375
3,"""นาย ณภัทร"" ปลื้มคนชมแชมป์ขึ้นปกนิตยสารแห่งปี ...",กลายเป็นดาราหนุ่มเนื้อหอมแฟนคลับแน่น กระแสมาแร...,2015-12-04 07:10:26,entertainment,https://www.matichon.co.th/entertainment/news_393,393
4,"คอแทบหัก! แม่ยกแห่คล้องพวงมาลัยักษ์ ""บอย ศิริช...",แสดงดีจนเป็นที่ถูกอกถูกใจแฟนคลับ จนได้รับพวงมา...,2015-12-05 05:26:20,entertainment,https://www.matichon.co.th/entertainment/news_445,445
...,...,...,...,...,...,...
17104,โบว์ ณัฏฐา แจ้งความ พล.ต.อ.ศรีวราห์ ถูกพาดพิง...,เมื่อวันที่ 5 ก.ค. ที่ สน.พญาไท น.ส.ณัฏฐา มหัท...,2018-07-05 13:25:45,politics,https://www.matichon.co.th/politics/news_1029607,1029607
17105,ภาพบรรยากาศ ขุดทางระบายน้ำ เร่งนำ 13 ชีวิตออกจ...,วันที่ 5 กรกฎาคม เจ้าหน้าที่ขุดทางระบายน้ำที่ด...,2018-07-05 13:33:10,region,https://www.matichon.co.th/region/news_1029619,1029619
17106,สนช.ผ่านพ.ร.บ.สงฆ์ 3 วาระรวด พระมหากษัตริย์ทรง...,"สนช.ผ่าน พ.ร.บ.สงฆ์ 3 วาระรวด ""วิษณุ"" แจงสาระส...",2018-07-05 13:33:27,politics,https://www.matichon.co.th/politics/news_1029636,1029636
17107,นานาทรรศนะเพิ่มค่าปรับหมอ 5ล้านบ. สกัดเบี้ยว...,หมายเหตุ – จากกรณีที่ กระทรวงศึกษาธิการ (ศธ.) ...,2018-07-05 13:53:26,education,https://www.matichon.co.th/education/news_1029668,1029668


In [4]:
## drop column except for article
df = df.drop(columns=['date', 'headline', 'url','id'])

## tokenize
df['article_tokens'] = df['article'].apply(my_tokenize)

df

,article,category,article_tokens
0,(2 ธ.ค.58) หนังสือพิมพ์อัล ริยาดของทางการซาอุด...,foreign,"[(, 2, , ธ.ค., 58, ), , หนังสือพิมพ์, อัล, ..."
1,"""บิ๊กป้อม"" แจง ครม. มีความพยายามยุยงปลุกปั่นให...",politics,"[บิ๊ก, ป้อม, , แจง, , ครม., , มี, ความพยายา..."
2,แม้จะทำหน้าที่ภรรยาที่ดีมาเฝ้าปอ – ทฤษฎี สหวงษ...,entertainment,"[แม้, จะ, ทำหน้าที่, ภรรยา, ที่, ดี, มา, เฝ้า,..."
3,กลายเป็นดาราหนุ่มเนื้อหอมแฟนคลับแน่น กระแสมาแร...,entertainment,"[กลายเป็น, ดารา, หนุ่ม, เนื้อ, หอม, แฟนคลับ, แ..."
4,แสดงดีจนเป็นที่ถูกอกถูกใจแฟนคลับ จนได้รับพวงมา...,entertainment,"[แสดง, ดี, จน, เป็นที่, ถูกอกถูกใจ, แฟนคลับ, ..."
...,...,...,...
17104,เมื่อวันที่ 5 ก.ค. ที่ สน.พญาไท น.ส.ณัฏฐา มหัท...,politics,"[เมื่อ, วันที่, , 5, , ก.ค., , ที่, , สน.,..."
17105,วันที่ 5 กรกฎาคม เจ้าหน้าที่ขุดทางระบายน้ำที่ด...,region,"[วันที่, , 5, , กรกฎาคม, , เจ้าหน้าที่, ขุด..."
17106,"สนช.ผ่าน พ.ร.บ.สงฆ์ 3 วาระรวด ""วิษณุ"" แจงสาระส...",politics,"[สนช, ., ผ่าน, , พ.ร.บ., สงฆ์, , 3, , วาระ,..."
17107,หมายเหตุ – จากกรณีที่ กระทรวงศึกษาธิการ (ศธ.) ...,education,"[หมายเหตุ, , –, , จาก, กรณี, ที่, , กระทรวง..."


# Fit model

how to use :
[https://radimrehurek.com/gensim/models/word2vec.html](https://radimrehurek.com/gensim/models/word2vec.html)

- input of `Word2Vec` must be **list of list of tokens**
- `vector_size` : Dimensionality of the word vectors (usually 100-300)
- `window` : Maximum distance between the current and predicted word within a sentence
- `min_count` : Ignores all words with total frequency lower than this
- `sg` : Training algorithm: 1 for skip-gram; otherwise CBOW. (skip-gram is recommended)
- `epoch` : Number of iterations (epochs) over the corpus

In [5]:
## fit
## it may take over 5 minutes
model = Word2Vec(sentences=df['article_tokens'], vector_size=100, window=5, min_count=3, sg=1, epochs=30)

# Word Embeddings

- `model.wv.most_similar(word, topn=xx)` gives the top N words with high cosine similarity 


In [6]:
model.wv.most_similar('อร่อย', topn=15)

[('รสชาติ', 0.8035041093826294),
 ('ลิ้มลอง', 0.7660192251205444),
 ('เมนู', 0.7526615858078003),
 ('กลมกล่อม', 0.7501031160354614),
 ('จุใจ', 0.7314376831054688),
 ('ถูกปาก', 0.7163810729980469),
 ('กรุบ', 0.7123017311096191),
 ('พาสต้า', 0.6868674159049988),
 ('คุณหนู', 0.6866104006767273),
 ('เครื่องเคียง', 0.6860358119010925),
 ('เย็นฉ่ำ', 0.6812074780464172),
 ('กระท้อน', 0.6786156892776489),
 ('จัดจ้าน', 0.677098274230957),
 ('Nespresso', 0.6757398247718811),
 ('เปรี้ยวหวาน', 0.6750460863113403)]

- `model.wv.similarity(word1, word2)` calculates the cosine similarity of the 2 words

In [7]:
## antonym is also similar word
model.wv.similarity('ผู้ชาย', 'ผู้หญิง')

0.805702

- `model.wv` is like a dictionary. `model.wv[word]` gives the vector of the word

In [8]:
model.wv['อร่อย']

array([ 0.31307328, -0.07921533, -0.3867272 ,  0.13724503,  0.24059764,
       -0.73763037, -0.5442555 ,  0.5338377 , -0.26995844, -0.29698443,
       -0.21065362, -0.10233144, -0.27221975,  0.2997619 , -0.18413778,
       -0.66272104,  0.20490527, -0.01256631,  0.20306937,  0.09954628,
       -0.39045006,  0.99344844,  0.04095479, -0.49357107,  0.22121052,
       -0.75070435, -0.91891277, -0.11065202, -0.40035784, -0.42389673,
        0.06553186, -0.35369262,  0.7488296 ,  0.4160489 , -0.31296515,
        0.2392835 , -0.04078358, -0.02962407, -0.40125602, -0.8883308 ,
       -0.41820616,  0.17129114, -0.65049976,  0.36274785, -0.05608656,
       -0.38620043,  0.4891837 , -0.5926844 ,  0.24443436,  0.74701077,
       -0.11308683, -0.681363  , -0.13900931, -0.55174935, -0.27999875,
        0.6628129 ,  0.62723684,  0.49221078, -0.5199675 , -0.5820469 ,
        0.37421325,  0.20049609, -0.5923961 , -0.18502884,  0.02285597,
       -0.26443905,  0.3510152 ,  0.840028  , -0.871073  , -0.54

- `model.wv.most_similar()` can add/subtract vectors
- use argument `positive` and `negative`, e.g. `model.wv.most_similar(positive=[w1, w2], negative=[w3])`

In [13]:
## ปักกิ่ง - จีน + ญี่ปุ่น
model.wv.most_similar(positive=['ปักกิ่ง', 'ญี่ปุ่น'], negative=['จีน'])

[('โตเกียว', 0.6219161748886108),
 ('กรุง', 0.5929952263832092),
 ('อาบูดาบี', 0.5402072668075562),
 ('ฮาเนดะ', 0.5307822227478027),
 ('แคนาดา', 0.5281808972358704),
 ('เตหะราน', 0.5262698531150818),
 ('อินโดนีเซีย', 0.5254713296890259),
 ('อัมมาน', 0.5177352428436279),
 ('ริยาด', 0.5120678544044495),
 ('นัตสึกิ', 0.5081430077552795)]

# save & load model

In [10]:
## save the model
## the same model is in `data` folder
model.save("word2vec_matichon.model")

In [11]:
## load model 
model_loaded = Word2Vec.load("word2vec_matichon.model")
model_loaded.wv.most_similar('สวย')

[('เซ็กซี่', 0.6698407530784607),
 ('ลุค', 0.6534721255302429),
 ('ลๆ', 0.6369643807411194),
 ('น่ารัก', 0.6328151822090149),
 ('เป็นสาว', 0.6288453936576843),
 ('สวยงาม', 0.6232518553733826),
 ('สะบึม', 0.6176999807357788),
 ('NAMU', 0.6087443828582764),
 ('บิกินี่', 0.6073794364929199),
 ('เอ้า!', 0.5966705083847046)]